## CDCS_MP

This notebook obtains and performs some image analysis on optical images taken under bright field (BF) imaging conditions from the ambench.nist.gov database.

In [1]:
import pandas as py             # a standard Python data processing library
import lxml.etree as et
from cdcs import CDCS
from urllib import request
from skimage import io

### 1. Importing records from CDCS
 - uses the CDCS REST API client to search for datasets related to the melt pool (MP) challenge within the ambench.nist.gov   instance
 - searches the returned datasets for tiff images containing the phrase "BF" and collects information about the corresponding track and case for each image
 - stores image names, image data, track, and case in a pandas dataframe

create a CDCS query instance and input a keyword

In [2]:
keyword='MP'
curator = CDCS('https://ambench.nist.gov/', username='') #accessing anonymously
df=curator.query(template='AM-Bench-2018',keyword=keyword) #searching for results with the phrase'MP'
df #display the dataframe

,id,template,workspace,user_id,title,xml_content,creation_date,last_modification_date,last_change_date,template_title
0,61e2e74036f033d46e0672b4,61e2e3e16f187f2844d7ae98,61e2e39ef79c150461de72ca,3,AMB2018-02-AMMT-625-TR6-MP.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/2...",2021-09-02T18:58:53.413000Z,2021-09-02T18:58:53.413000Z,2021-09-02T18:58:53.735000Z,AM-Bench-2018
1,61e2e71747ed0ce2c83c23ac,61e2e3e16f187f2844d7ae98,61e2e39ef79c150461de72ca,3,AMB2018-02-AMMT-625-TR5-MP.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/2...",2021-09-02T18:54:53.573000Z,2021-09-02T18:54:53.573000Z,2021-09-02T18:54:53.920000Z,AM-Bench-2018
2,61e2e700f51a62700be8516a,61e2e3e16f187f2844d7ae98,61e2e39ef79c150461de72ca,3,AMB2018-02-AMMT-625-TR8-MP.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/2...",2021-09-02T19:00:55.329000Z,2021-09-02T19:00:55.329000Z,2021-09-02T19:00:55.632000Z,AM-Bench-2018
3,61e2e6a3e0ade599482c5b77,61e2e3e16f187f2844d7ae98,61e2e39ef79c150461de72ca,3,AMB2018-02-AMMT-625-TR2-MP.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/20...",2021-09-02T18:58:21.909000Z,2021-09-02T18:58:21.909000Z,2021-09-02T18:58:22.210000Z,AM-Bench-2018
4,61e2e653b8edd76d53d3d54f,61e2e3e16f187f2844d7ae98,61e2e39ef79c150461de72ca,3,AMB2018-02-AMMT-625-TR3-MP.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/2...",2021-09-02T19:00:09.818000Z,2021-09-02T19:00:09.818000Z,2021-09-02T19:00:10.144000Z,AM-Bench-2018
5,61e2e63bf51a62700be850ec,61e2e3e16f187f2844d7ae98,61e2e39ef79c150461de72ca,3,AMB2018-02-AMMT-625-TR10-MP.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/2...",2021-09-02T18:58:34.704000Z,2021-09-02T18:58:34.704000Z,2021-09-02T18:58:35.022000Z,AM-Bench-2018
6,61e2e580d682ae6fb2db87af,61e2e3e16f187f2844d7ae98,61e2e39ef79c150461de72ca,3,AMB2018-02-AMMT-625-TR4-MP.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/2...",2021-09-02T19:00:37.749000Z,2021-09-02T19:00:37.749000Z,2021-09-02T19:00:38.129000Z,AM-Bench-2018
7,61e2e52fe0ade599482c58ba,61e2e3e16f187f2844d7ae98,61e2e39ef79c150461de72ca,3,AMB2018-02-AMMT-625-TR9-MP.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/2...",2021-09-02T18:54:40.113000Z,2021-09-02T18:54:40.113000Z,2021-09-02T18:54:40.430000Z,AM-Bench-2018
8,61e2e4c4b8edd76d53d3cb8e,61e2e3e16f187f2844d7ae98,61e2e39ef79c150461de72ca,3,AMB2018-02-AMMT-625-TR1-MP.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/20...",2021-09-02T18:59:31.536000Z,2021-09-02T18:59:31.536000Z,2021-09-02T18:59:31.833000Z,AM-Bench-2018
9,61e2e46326f0a0b0e709c1df,61e2e3e16f187f2844d7ae98,61e2e39ef79c150461de72ca,3,AMB2018-02-AMMT-625-TR7-MP.xml,"<AMBench2018LB xmlns:xsi=""http://www.w3.org/2...",2021-09-02T18:56:09.153000Z,2021-09-02T18:56:09.153000Z,2021-09-02T18:56:09.466000Z,AM-Bench-2018


#### xml_url_find:
Parameters:
- XML file for one of the ambench.nist.gov datasets in string format
- search phrase contained in the file name
- file type

Returns:
- a list containing (0) name of a file, (1) the download url for the file, (2) laser track number, (3) case 

In [3]:
def xml_url_find(xml,searchphrase,mtype):
    root=et.fromstring(xml)
    caseid=root.find('.//TraceID')[0].tag[5]
    track=root.find('.//TrackNumber')
    for element in root.iter('downloadURL'):
        u=request.urlopen(element.text)
        if searchphrase in u.info().get_filename() and mtype in u.info().get_content_type(): #looking for searchphrase in
            #filename and mtype in file type
            name=u.info().get_filename()
            url=element.text
            return [name,url,track.text,caseid]

In [7]:
searchphrase='BF' #returns a list of scikit images
filename=[]
result=[]
trace=[]
case=[]
for index in df.index:
    res=xml_url_find(df.iloc[index]['xml_content'],'BF','image/tiff') #giving xml_url_find each xml string and searching for
    #tiff files with 'BF' in their name 
    name=res[0]
    url=res[1]
    track=res[2]
    c=res[3]
    print(url)
    if url==None:
        continue
    else:
        filename.append(name)
        case.append(c)
        result.append(io.imread(url)) #reading in the urls using scikit
        trace.append(track)
dfres=py.DataFrame({'name':filename,'image':result,'track':trace,'case':case}) #stores images and image names in a dataframe
dfres

https://ambench.nist.gov/rest/blob/download/61e2e72d26f0a0b0e709c949
https://ambench.nist.gov/rest/blob/download/61e2e708b8edd76d53d3d8a0
https://ambench.nist.gov/rest/blob/download/61e2e6ede0ade599482c5c20
https://ambench.nist.gov/rest/blob/download/61e2e69426f0a0b0e709c6a4
https://ambench.nist.gov/rest/blob/download/61e2e63fb8edd76d53d3d4e1
https://ambench.nist.gov/rest/blob/download/61e2e6287774703da98d09e4
https://ambench.nist.gov/rest/blob/download/61e2e56df51a62700be8507a
https://ambench.nist.gov/rest/blob/download/61e2e51cdaebe69a45e91632
https://ambench.nist.gov/rest/blob/download/61e2e4b4f51a62700be84de7
https://ambench.nist.gov/rest/blob/download/61e2e44f47ed0ce2c83c2089


,name,image,track,case
0,AMB18-02-AMMT-Trace06-50xBF-ZStk-EDF.tif,"[[[198, 199, 201], [196, 200, 201], [193, 193,...",6,A
1,AMB18-02-AMMT-Trace05-50xBF-ZStk-EDF.tif,"[[[175, 182, 189], [185, 193, 193], [183, 197,...",5,A
2,AMB18-02-AMMT-Trace08-50xBF-ZStk-EDF.tif,"[[[198, 202, 207], [197, 203, 200], [201, 202,...",8,B
3,AMB18-02-AMMT-Trace02-50xBF-ZStk-EDF.tif,"[[[183, 189, 197], [185, 186, 193], [190, 192,...",2,C
4,AMB18-02-AMMT-Trace03-50xBF-ZStk-EDF.tif,"[[[178, 179, 174], [184, 185, 174], [187, 184,...",3,C
5,AMB18-02-AMMT-Trace10-50xBF-ZStk-EDF.tif,"[[[195, 196, 205], [197, 204, 210], [202, 205,...",10,B
6,AMB18-02-AMMT-Trace04-50xBF-ZStk-EDF.tif,"[[[180, 186, 195], [185, 181, 197], [185, 181,...",4,C
7,AMB18-02-AMMT-Trace09-50xBF-ZStk-EDF.tif,"[[[202, 202, 206], [206, 206, 206], [203, 206,...",9,B
8,AMB18-02-AMMT-Trace01-50xBF-ZStk-EDF.tif,"[[[191, 197, 202], [195, 201, 204], [190, 197,...",1,C
9,AMB18-02-AMMT-Trace07-50xBF-ZStk-EDF.tif,"[[[196, 199, 199], [197, 204, 201], [199, 204,...",7,A


### 2. Image visualization and analysis
- using scikit-image to perform image analysis and matplotlib to display images
- uses the felzenszwalb image segmentation algorithm to help obtain widths and depths of the melt pools
- displays plots of width and depth
- stores measurements in a csv file

In [5]:
import skimage
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#### draw_box_opt
This method (1) crops the melt pool images (to reduce runtime), (2) applies the felzenszwalb segmentation algorithm to the image, (3) removes smaller regions (eliminate some background noise), (4) acquires a table of center coordinates for the remaining regions, (5) measures the distances from centers to the center of the image, (6) finds the index number of the closest region, (7) returns information about the closes region

Parameters:
- image to be analyzed

Returns:
- list of RegionProperties related to the melt pool region

In [6]:
from skimage import filters
from skimage import segmentation
from skimage import morphology
from skimage import measure
import math
def draw_box_opt(image):
    cropim=image[900:1800,800:3400] #cropping so that there's less pixels to cover - incredibly slow if left at original size
    segments=segmentation.felzenszwalb(cropim,scale=270,sigma=0.8,min_size=1000)
    isolateim=morphology.remove_small_objects(segments,100000)
    center=measure.regionprops_table(isolateim,properties=['centroid']) #table of information about the center points of regions
    distances=[]
    for n in range(len(center['centroid-0'])):
        distances.append(math.dist([center['centroid-0'][n],center['centroid-1'][n]],[450,1300]))
    index=distances.index(min(distances)) #Looking for the region closest to the center of the image - should be the melt pool
    object_features=measure.regionprops(isolateim)
    return object_features[index] 

In [ ]:
depths=[]
widths=[]
bboxes=[]
for im in dfres['image']:
    objf=draw_box_opt(im)
    bboxes.append(objf.bbox)
    minr,minc,maxr,maxc=objf.bbox #using the bbox drawn around the melt pool to find width and depth
    widths.append((maxc-minc)*(0.062)) #using the conversion given in the CDCS comment above the images to convert to mm (1 pixel=0.062 microns)
    depths.append((maxr-minr)*(0.062))
dat={'width':widths,'depth':depths,'track':trace,'case':case}

In [ ]:
dfmp=py.DataFrame(dat)

In [ ]:
dfmp

Bar plot of melt pool widths

In [ ]:
dfmp.plot(y='width',x='track',kind='bar')

Bar plot of melt pool depths

In [ ]:
dfmp.plot(y='depth',x='track',kind='bar')

Example of what the image looks like with the bbox marked

In [ ]:
fig, ax = plt.subplots()
minr,minc,maxr,maxc=bboxes[4]
ax.imshow(result[4][900:1800,800:3400])
bx = (minc, maxc, maxc, minc, minc)
by = (minr, minr, maxr, maxr, minr)
ax.plot(bx, by, '-b', linewidth=2.5)
plt.show()

Putting the calculated measurements into a csv file and saving it

In [ ]:
dfmp.to_csv('MP_measurements.csv',index=False)